# 02 - Property Tax Analysis

Deep dive into property tax structures across Eastchester-area municipalities.

## Overview
1. Understanding tax components (RAR, rates, jurisdictions)
2. Calculate taxes for sample properties
3. Compare effective tax rates
4. Analyze the Eastchester/Bronxville/Tuckahoe relationship


In [ ]:
# Setup
import sys
sys.path.insert(0, '../src')

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from config import MUNICIPALITIES, EASTCHESTER_AREA, get_municipality
from models.tax_calculator import TaxCalculator

sns.set_theme(style='whitegrid')
calc = TaxCalculator()
print("Setup complete!")


## 1. Understanding Residential Assessment Ratios (RAR)

The RAR determines what percentage of market value is used for tax assessment.


In [ ]:
# Show RAR for each municipality
print("Residential Assessment Ratios (RAR):")
print("=" * 60)
print(f"{'Municipality':<30} {'RAR':>10} {'$1M Assessment':>15}")
print("-" * 60)

for key, muni in MUNICIPALITIES.items():
    assessed = 1_000_000 * muni.rar
    rar_pct = muni.rar * 100
    print(f"{muni.name:<30} {rar_pct:>9.2f}% ${assessed:>13,.0f}")

print("\n💡 Note: Low RAR ≠ Low Taxes! Rates adjust to compensate.")


## 2. Tax Calculation: $1M Home Comparison


In [ ]:
# Calculate taxes for a $1M home across all municipalities
market_value = 1_000_000

results = []
for key in MUNICIPALITIES.keys():
    breakdown = calc.calculate_from_market_value(market_value, key)
    results.append({
        'Municipality': breakdown.municipality,
        'Annual Tax': breakdown.total,
        'Monthly': breakdown.total / 12,
        'Effective Rate': breakdown.effective_rate,
        'School %': breakdown.school_percentage,
    })

tax_df = pd.DataFrame(results).sort_values('Annual Tax')
print(f"Tax Comparison: ${market_value:,} Home\n")
tax_df


In [ ]:
# Visualize tax comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Annual tax
tax_sorted = tax_df.sort_values('Annual Tax', ascending=True)
axes[0].barh(tax_sorted['Municipality'], tax_sorted['Annual Tax'], color='steelblue')
axes[0].set_xlabel('Annual Tax ($)')
axes[0].set_title(f'Annual Property Tax for ${market_value:,} Home')

# Effective rate
rate_sorted = tax_df.sort_values('Effective Rate', ascending=True)
axes[1].barh(rate_sorted['Municipality'], rate_sorted['Effective Rate'], color='coral')
axes[1].set_xlabel('Effective Tax Rate (%)')
axes[1].set_title('Effective Tax Rate')

plt.tight_layout()
plt.show()


## 3. Eastchester Area Detailed Breakdown


In [ ]:
# Detailed breakdown for Eastchester area
print("Detailed Tax Breakdown: Eastchester Area")
print("=" * 60)

for muni in ['bronxville', 'eastchester_unincorp', 'tuckahoe']:
    breakdown = calc.calculate_from_market_value(market_value, muni)
    print(breakdown)
    print()


In [ ]:
# Stacked bar chart of tax components
components = []
for key in ['bronxville', 'eastchester_unincorp', 'tuckahoe', 'scarsdale', 'larchmont']:
    breakdown = calc.calculate_from_market_value(market_value, key)
    components.append({
        'Municipality': MUNICIPALITIES[key].name,
        'School': breakdown.school_tax,
        'County': breakdown.county_tax,
        'Town': breakdown.town_tax,
        'Village': breakdown.village_tax,
        'Fire/Library': breakdown.fire_district_tax + breakdown.library_tax,
    })

comp_df = pd.DataFrame(components).set_index('Municipality')

fig, ax = plt.subplots(figsize=(12, 6))
comp_df.plot(kind='bar', stacked=True, ax=ax, colormap='Set2')
ax.set_ylabel('Annual Tax ($)')
ax.set_title(f'Tax Component Breakdown for ${market_value:,} Home')
ax.legend(title='Component', bbox_to_anchor=(1.02, 1))
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print("\n📊 School taxes typically make up 60-80% of total property tax!")
